In [38]:
import pandas as pd
import simfin as sf
import numpy as np
import scipy as sp
import datetime as date
from datetime import datetime
from simfin.names import *
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as scst

In [39]:
#collect data from api
sf.set_api_key('PLEbHyzQgm98TrvHFejzBwikq1mKiJob ')
sf.set_data_dir('~/simfin_data/')
sf.set_data_dir('C:/Users/suher/Python/Springboard/Capstone 2')
figures_and_ratios = sf.load_derived(variant = 'quarterly',market = 'us')
sector_industry = sf.load_industries()
companies = sf.load_companies(market='us')
historical_prices = sf.load_shareprices(variant = 'daily', market = 'us')
price_ratios = sf.load_derived_shareprices(variant='daily', market='us')


Dataset "us-derived-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "industries" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-companies" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-derived-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!


In [40]:
hist_price = historical_prices.drop(['Low','High','Adj. Close','Dividend','Shares Outstanding'],axis=1)
sp500 = pd.read_csv('constituents_csv.csv')

In [41]:
figures_and_ratios['FCF/EBITDA'] = figures_and_ratios['Free Cash Flow']/figures_and_ratios['EBITDA']

In [42]:
fund_features = figures_and_ratios[['SimFinId','FCF/EBITDA','Liabilities to Equity Ratio','Earnings Per Share, Diluted']]
p_ratios = price_ratios[['SimFinId','Price to Earnings Ratio (quarterly)','Price to Book Value']]
fund_features=fund_features.reset_index()
p_ratios = p_ratios.reset_index()

In [43]:
df = pd.merge(p_ratios,fund_features.drop('SimFinId',axis=1),how='left',left_on=['Ticker','Date'], right_on=['Ticker','Report Date'])
df['Ticker'] = df['Ticker'].astype('string') 
df_clean = df.dropna()
df_clean['eps_growth'] = df_clean['Earnings Per Share, Diluted'].pct_change()
df_clean=df_clean.reset_index()
df_clean['peg'] = df_clean['eps_growth']/df_clean['Earnings Per Share, Diluted']

<ipython-input-43-cda661c41d5b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['eps_growth'] = df_clean['Earnings Per Share, Diluted'].pct_change()


In [44]:
ticker_list = df_clean.Ticker.unique()
droplist = []

In [45]:
for i in ticker_list: 
    row = df_clean[df_clean['Ticker'] == i].index[0]
    droplist.append(row)
    
df_clean = df_clean.drop(droplist)

In [46]:
fundamentals = df_clean.drop(['Report Date','Earnings Per Share, Diluted','eps_growth'],axis=1)
fundamentals = fundamentals.drop('index',axis=1)
fundamentals.columns = ['Ticker','Date','SimFinId','pe','pb','febitda','de','peg']
p_ret = hist_price.reset_index()
p_ret['open_ret'] = p_ret['Open'].pct_change()
p_ret['close_ret'] = p_ret['Close'].pct_change()

In [47]:
companies_id = companies[['SimFinId','IndustryId']]
companies_sec_ind = pd.merge(companies_id,sector_industry,how='left',left_on='IndustryId',right_on='IndustryId')
companies_sec_ind = companies_sec_ind[['SimFinId','Sector','Industry']]
fundamentals = pd.merge(fundamentals,companies_sec_ind,how='left',left_on='SimFinId',right_on='SimFinId')
fundamentals = fundamentals.dropna()
historical_prices_res = historical_prices.reset_index()
ticker_list = list(fundamentals.Ticker.unique())


In [48]:
fundamentals = pd.merge(fundamentals,historical_prices_res[['Ticker','Date','Close']],how='left',left_on=['Ticker','Date'],right_on=['Ticker','Date'])
fundamentals.set_index(['Ticker','Date'],inplace=True)
sp500['check'] = 1


fd_test = fundamentals.reset_index()

fd_test = pd.merge(fd_test,sp500[['Symbol','check']],how="left",left_on="Ticker",right_on="Symbol")
fd_test = fd_test[fd_test['check'] == 1]
fd_test.drop(['Symbol','check'],axis=1,inplace=True)

phist=historical_prices.reset_index()

phist = phist[['Ticker','Date','Close']]

phist['ret'] = phist['Close'].pct_change()

phist['a_ret'] = (phist['ret']*252).rolling(252).mean()
phist['a_std'] = phist['ret'].rolling(252).std() * np.sqrt(252)
phist['sharpe'] = phist['a_ret']/phist['a_std']


In [52]:
features = pd.merge(fd_test,phist,how='left',left_on=['Ticker','Date'],right_on=['Ticker','Date'])

In [54]:
features.drop('Close_y',axis=1,inplace=True)
features.head()

,Ticker,Date,SimFinId,pe,pb,febitda,de,peg,Sector,Industry,Close_x,ret,a_ret,a_std,sharpe
0,A,2011-10-31,45846,39.09200,3.05828,1.111702,1.09847,0.617936,Healthcare,Medical Diagnostics & Research,37.07,-0.093421,0.169070,0.456239,0.370574
1,A,2012-01-31,45846,50.84644,3.41453,0.381381,1.02245,-0.308984,Healthcare,Medical Diagnostics & Research,42.47,-0.015987,0.134647,0.488014,0.275907
2,A,2012-04-30,45846,63.82017,3.25561,1.077562,0.99217,0.142199,Healthcare,Medical Diagnostics & Research,42.18,-0.008929,-0.055932,0.472750,-0.118312
3,A,2012-07-31,45846,52.25459,2.81604,0.519774,1.00226,-0.064432,Healthcare,Medical Diagnostics & Research,38.29,0.002881,0.050150,0.475088,0.105560
4,A,2012-10-31,45846,51.54123,2.56649,0.645161,1.03202,0.622079,Healthcare,Medical Diagnostics & Research,35.99,0.001391,-0.068796,0.341985,-0.201166


In [70]:
features = features.set_index(['Ticker','Date'])

KeyError: "None of ['Ticker', 'Date'] are in the columns"

In [72]:
features

SimFinId         pe        pb   febitda       de       peg  \
Ticker Date                                                                     
A      2011-10-31     45846   39.09200   3.05828  1.111702  1.09847  0.617936   
       2012-01-31     45846   50.84644   3.41453  0.381381  1.02245 -0.308984   
       2012-04-30     45846   63.82017   3.25561  1.077562  0.99217  0.142199   
       2012-07-31     45846   52.25459   2.81604  0.519774  1.00226 -0.064432   
       2012-10-31     45846   51.54123   2.56649  0.645161  1.03202  0.622079   
...                     ...        ...       ...       ...      ...       ...   
ZTS    2018-12-31    378251  118.81925  19.57840  1.152700  3.93227  0.595683   
       2019-09-30    378251  160.79162  24.81661  0.932936  3.20911  0.292304   
       2019-12-31    378251  146.04349  23.68205  0.975369  3.26329 -0.138628   
       2020-03-31    378251  145.98219  20.77950  0.801233  3.18743  0.118254   
       2020-06-30    378251  154.08091  23.75068  0.947289  3.37529 -0.135779   

                       Sector                        Industry  Close_x  \
Ticker Date                                                              
A      2011-10-31  Healthcare  Medical Diagnostics & Research    37.07   
       2012-01-31  Healthcare  Medical Diagnostics & Research    42.47   
       2012-04-30  Healthcare  Medical Diagnostics & Research    42.18   
       2012-07-31  Healthcare  Medical Diagnostics & Research    38.29   
       2012-10-31  Healthcare  Medical Diagnostics & Research    35.99   
...                       ...                             ...      ...   
ZTS    2018-12-31  Healthcare              Drug Manufacturers    85.54   
       2019-09-30  Healthcare              Drug Manufacturers   124.59   
       2019-12-31  Healthcare              Drug Manufacturers   132.35   
       2020-03-31  Healthcare              Drug Manufacturers   117.69   
       2020-06-30  Healthcare              Drug Manufacturers   137.04   

                        ret     a_ret     a_std    sharpe  
Ticker Date                                                
A      2011-10-31 -0.093421  0.169070  0.456239  0.370574  
       2012-01-31 -0.015987  0.134647  0.488014  0.275907  
       2012-04-30 -0.008929 -0.055932  0.472750 -0.118312  
       2012-07-31  0.002881  0.050150  0.475088  0.105560  
       2012-10-31  0.001391 -0.068796  0.341985 -0.201166  
...                     ...       ...       ...       ...  
ZTS    2018-12-31  0.012428  0.194316  0.236686  0.820989  
       2019-09-30  0.005569  0.339252  0.236059  1.437153  
       2019-12-31  0.001059  0.458104  0.209791  2.183615  
       2020-03-31  0.008656  0.206542  0.343240  0.601743  
       2020-06-30  0.038339  0.264657  0.392505  0.674275  

[9094 rows x 13 columns]